In [1]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset
from dataset import ThyroidNodules
from torch.utils.data import DataLoader
from glob import glob
from trainer import Trainer
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    jaccard_score,
    precision_score,
    recall_score,
)
from eval import compute_loss,DiceLoss
from timeit import default_timer as timer
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils import weight_norm
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
%load_ext autoreload
%autoreload 2

In [2]:
# Batch size and image dimensions
BATCH_SIZE = 16
HEIGHT = 128
WIDTH = 128

# Hyperparameters
H = HEIGHT
W = WIDTH
DEFAULT_LR = 0.001

train_x= sorted(glob(os.path.join("/Users/eloise-em/Documents/Haris Ghafoor Archive/Research and Development/RnD/Thyroid Dataset/tn3k/trainval-image",'*')))
train_y= sorted(glob(os.path.join("/Users/eloise-em/Documents/Haris Ghafoor Archive/Research and Development/RnD/Thyroid Dataset/tn3k/trainval-mask",'*')))

valid_x= sorted(glob(os.path.join("/Users/eloise-em/Documents/Haris Ghafoor Archive/Research and Development/RnD/Thyroid Dataset/tn3k/test-image",'*')))
valid_y= sorted(glob(os.path.join("/Users/eloise-em/Documents/Haris Ghafoor Archive/Research and Development/RnD/Thyroid Dataset/tn3k/test-mask",'*')))

# valid_x = "/Users/eloise-em/Documents/Haris Ghafoor Archive/Research and Development/RnD/Thyroid Dataset/tn3k/test-image"
# valid_y = "/Users/eloise-em/Documents/Haris Ghafoor Archive/Research and Development/RnD/Thyroid Dataset/tn3k/test-mask"
# Create datasets and dataloaders for this fold
train_dataset = ThyroidNodules(
    train_x,
    train_y,
    image_size=(H, W),
)

valid_dataset = ThyroidNodules(
    valid_x,
    valid_y,
    image_size=(H, W),
)

train_loader = DataLoader(
    dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2
)
valid_loader = DataLoader(
    dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2
)
# device = torch.device(args.device)
# print("Device:", device)

# model = AttentionUNetppGradual()
# model = model.to(device)

# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, "min", patience=10, verbose=True
# )

In [3]:
trainer = Trainer(42)

Using Apple MPS


In [4]:
def calculate_metrics(y_true, y_pred, threshold):
    # Ground truth
    y_true = y_true.cpu().numpy()
    y_true = y_true > threshold
    y_true = y_true.astype(np.uint8)
    y_true = y_true.reshape(-1)

    # Prediction
    # y_pred = torch.sigmoid(y_pred)
    y_pred = y_pred.detach().cpu().numpy()
    y_pred = y_pred > threshold
    y_pred = y_pred.astype(np.uint8)
    y_pred = y_pred.reshape(-1)

    score_jaccard = jaccard_score(y_true, y_pred)
    score_f1 = f1_score(y_true, y_pred)
    score_recall = recall_score(y_true, y_pred)
    score_precision = precision_score(y_true, y_pred)
    score_acc = accuracy_score(y_true, y_pred)

    return [score_jaccard, score_f1, score_recall, score_precision, score_acc]

In [5]:
trainer = Trainer(seed=7)
input = next(iter(trainer.train_loader))[0].to(trainer.device)
output = trainer.model(input)
target = next(iter(trainer.train_loader))[1].to(trainer.device)
loss = compute_loss(output, target)
print("output", output.shape)
print("target", target.shape)
print("loss", loss)


Using Apple MPS
output torch.Size([32, 1, 256, 256])
target torch.Size([32, 1, 256, 256])
loss tensor(1.7495, device='mps:0', grad_fn=<AddBackward0>)


In [6]:
model = trainer.model
checkpoint = torch.load("/Users/eloise-em/Documents/GitHub/Self-Gradual-Supervised-Learning-Network-for-Brain-Tumor-Segmentation/model_best.pth.tar")
model.load_state_dict(checkpoint["state_dict"])
model.eval()

Unet(
  (MaxPool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv1): ConvBlock(
    (conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (Conv2): ConvBlock(
    (conv): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  

In [7]:
# dict_results = trainer.predict(model = model)
# results = {"metrics": []}
# results = np.array()
dice_loss = 0
count = []
for x,y in trainer.test_dataset:
    # print(x.shape,y.shape)
    images = Variable(x.unsqueeze(0)).to(trainer.device)
    labels = Variable(y, requires_grad=False).to(trainer.device)
    out = model(images).squeeze(0)
    out = F.sigmoid(out)
    dice_loss = DiceLoss()(out, labels)
    dice_loss = 1 - dice_loss.item()
    dice_loss+= dice_loss
    # print(images.shape,labels.shape,out.shape)
    results = calculate_metrics(y_true=labels, y_pred=out, threshold=0.1)
    results+= np.array(results)
    # break
results = results / len(trainer.test_dataset)
dice_loss = dice_loss / len(trainer.test_dataset)

/Users/eloise-em/miniconda3/envs/ssl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/eloise-em/miniconda3/envs/ssl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/eloise-em/miniconda3/envs/ssl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [ ]:
for x,y in trainer.test_dataset:
    print(x.shape,y.shape)
    images = Variable(x.unsqueeze(0)).to(trainer.device)
    labels = Variable(y, requires_grad=False).to(trainer.device)
    out = model(images).squeeze(0)
    out = F.sigmoid(out)
    print(images.shape,labels.shape,out.shape)
    break

torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 256, 256]) torch.Size([1, 256, 256])
